In [31]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
dvfs_list = ['1', '0c00', '0d00', '0e00', 
             '0f00', '1000', '1100', '1200', 
             '1300', '1400', '1500', '1600', '1700', '1800', '1900', '1a00']
dvfs_dict = {
    "0x0c00" :  1.2,
    "0x0d00" :  1.3,
    "0x0e00" :  1.4,
    "0x0f00" :  1.5,
    "0x1000" : 1.6,
    "0x1100" : 1.7,
    "0x1200" : 1.8,
    "0x1300" : 1.9,
    "0x1400" : 2.0,
    "0x1500" : 2.1,
    "0x1600" : 2.2,
    "0x1700" : 2.3,
    "0x1800" : 2.4,
    "0x1900" : 2.5,
    "0x1a00" : 2.6,
    "0x1b00" : 2.7,
    "0x1c00" : 2.8,
    "0x1d00" : 2.9,
}

df_dict = {
    'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 
    
    'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [],'c3Cnt': [], 'c6Cnt': [], 
    'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
    'erxBytes': [], 'etxBytes':[],
    
    'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 
    'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 
    'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 
    'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 
    'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 

    'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 
    'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 
    'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 
    'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 
    'SourcebusyTime_%': [], 'SourcebackPressuredTime_%': [], 

    'MappernumRecordsInPerSecond_avg': [], 
    'MappernumRecordsInPerSecond_std': [], 'MappernumRecordsOutPerSecond_avg': [], 
    'MappernumRecordsOutPerSecond_std': [], 'MapperbusyTimeMsPerSecond_avg': [], 
    'MapperbusyTimeMsPerSecond_std': [], 'MapperbackPressuredTimeMsPerSecond_avg': [], 
    'MapperbackPressuredTimeMsPerSecond_std': [],
    'MapperbusyTime_%': [], 'MapperbackPressuredTime_%': []
}

print(df_dict)
print("*****************************************************************")

def parseFile(rate, itr, dvfs, policy, i):
    file=f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/summary.csv"
    print(file)
    df_dict['i'].append(i)
    df_dict['itr'].append(itr)
    if '0x'+dvfs in dvfs_dict:
        df_dict['dvfs'].append(dvfs_dict['0x'+dvfs])
    else:
        df_dict['dvfs'].append(dvfs)
    df_dict['policy'].append(policy)
    df_dict['rate'].append(rate)
        
    df = pd.read_csv(file)
    df = df[df.columns.drop(list(df.filter(regex='Cnt')))]
    df = df[df.columns.drop(list(df.filter(regex='Bytes')))]
    #print(df)
    
    dff = df[df['name'].str.contains('Sink')]
    dff.columns = 'Sink' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Source')]
    dff.columns = 'Source' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    dff = df[df['name'].str.contains('Mapper')]
    dff.columns = 'Mapper' + dff.columns
    cols = dff.columns
    for col in cols[2:]:
        df_dict[col].append(dff.mean(numeric_only=True)[col])

    jfile = f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/server2_rapl.log"
    with open(jfile) as file:
        lines = [float(line.rstrip()) for line in file]
        print(len(lines))
        df_dict['watts_avg'].append(float(round(np.mean(lines[300:500]), 2)))
        df_dict['watts_std'].append(float(round(np.std(lines[300:500]), 2)))
        #print(np.std(lines[300:500]))
        
    jfile = f"/home/han/github/logs/query1_cores16_frate{rate}_600000_fbuff-1_itr{itr}_{policy}dvfs{dvfs}_repeat{i}/stats.csv"
    with open(jfile) as file:
        poll = []
        c1 = []
        c1e = []
        c3 = []
        c6 = []
        rxp = []
        rxb = []
        txp = []
        txb = []
        erxb = []
        etxb = []
        for line in file:
            ll = [int(a) for a in line.strip().split(',')]
            poll.append(ll[0])
            c1.append(ll[1])
            c1e.append(ll[2])
            c3.append(ll[3])
            c6.append(ll[4])
            rxp.append(ll[5])
            rxb.append(ll[6])
            txp.append(ll[7])
            txb.append(ll[8])
            erxb.append(ll[9])
            etxb.append(ll[10])
        df_dict['pollCnt'].append(np.sum(poll[30:50]))
        df_dict['c1Cnt'].append(np.sum(c1[30:50]))
        df_dict['c1eCnt'].append(np.sum(c1e[30:50]))
        df_dict['c3Cnt'].append(np.sum(c3[30:50]))
        df_dict['c6Cnt'].append(np.sum(c6[30:50]))
        #'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [],
        df_dict['rxPackets'].append(np.sum(rxp[30:50]))
        df_dict['rxBytes'].append(np.sum(rxb[30:50]))        
        df_dict['txPackets'].append(np.sum(txp[30:50]))
        df_dict['txBytes'].append(np.sum(txb[30:50]))
        df_dict['erxBytes'].append(np.sum(erxb[30:50]))
        print(np.mean(erxb[30:50]), np.std(erxb[30:50]))
        df_dict['etxBytes'].append(np.sum(etxb[30:50]))
  

{'i': [], 'itr': [], 'dvfs': [], 'rate': [], 'policy': [], 'watts_avg': [], 'watts_std': [], 'pollCnt': [], 'c1Cnt': [], 'c1eCnt': [], 'c3Cnt': [], 'c6Cnt': [], 'rxPackets': [], 'rxBytes': [], 'txPackets': [], 'txBytes': [], 'erxBytes': [], 'etxBytes': [], 'SinknumRecordsInPerSecond_avg': [], 'SinknumRecordsInPerSecond_std': [], 'SinknumRecordsOutPerSecond_avg': [], 'SinknumRecordsOutPerSecond_std': [], 'SinkbusyTimeMsPerSecond_avg': [], 'SinkbusyTimeMsPerSecond_std': [], 'SinkbackPressuredTimeMsPerSecond_avg': [], 'SinkbackPressuredTimeMsPerSecond_std': [], 'SinkbusyTime_%': [], 'SinkbackPressuredTime_%': [], 'SourcenumRecordsInPerSecond_avg': [], 'SourcenumRecordsInPerSecond_std': [], 'SourcenumRecordsOutPerSecond_avg': [], 'SourcenumRecordsOutPerSecond_std': [], 'SourcebusyTimeMsPerSecond_avg': [], 'SourcebusyTimeMsPerSecond_std': [], 'SourcebackPressuredTimeMsPerSecond_avg': [], 'SourcebackPressuredTimeMsPerSecond_std': [], 'SourcebusyTime_%': [], 'SourcebackPressuredTime_%': [], '

In [42]:
itr=1
mdvfs=["0c00", "0d00", "0e00", "0f00", "1000", "1100", "1200", "1300", "1400", "1500", "1600", "1700", "1800", "1900", "1a00"]
nrepeat = 1

for dvfs in mdvfs:
    parseFile(300000, 1, dvfs, "userspace", 0)
print(df_dict)

/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_userspacedvfs0c00_repeat0/summary.csv
722
57127.55 3344.2241622684323
/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_userspacedvfs0d00_repeat0/summary.csv
716
57131.7 5224.937110626308
/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_userspacedvfs0e00_repeat0/summary.csv
719
68380.15 8120.725375697667
/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_userspacedvfs0f00_repeat0/summary.csv
715
62738.3 12089.978664580016
/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_userspacedvfs1000_repeat0/summary.csv
714
60778.2 11936.987273177434
/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_userspacedvfs1100_repeat0/summary.csv
743
58188.6 6125.747827000389
/home/han/github/logs/query1_cores16_frate300000_600000_fbuff-1_itr1_userspacedvfs1200_repeat0/summary.csv
714
55946.45 4105.329834191159
/home/han/github/logs/query1_cores1

In [45]:
d = pd.DataFrame(df_dict)
print(d.columns)
d0 = d[(d.i == 0)]
#print(d.groupby(i))
d0[['i', 'rate', 'dvfs', 
   'SourcenumRecordsOutPerSecond_avg', 'SourcenumRecordsOutPerSecond_std', 'MapperbusyTimeMsPerSecond_avg',
   'watts_avg', 'rxBytes', 'txBytes']]

Index(['i', 'itr', 'dvfs', 'rate', 'policy', 'watts_avg', 'watts_std',
       'pollCnt', 'c1Cnt', 'c1eCnt', 'c3Cnt', 'c6Cnt', 'rxPackets', 'rxBytes',
       'txPackets', 'txBytes', 'erxBytes', 'etxBytes',
       'SinknumRecordsInPerSecond_avg', 'SinknumRecordsInPerSecond_std',
       'SinknumRecordsOutPerSecond_avg', 'SinknumRecordsOutPerSecond_std',
       'SinkbusyTimeMsPerSecond_avg', 'SinkbusyTimeMsPerSecond_std',
       'SinkbackPressuredTimeMsPerSecond_avg',
       'SinkbackPressuredTimeMsPerSecond_std', 'SinkbusyTime_%',
       'SinkbackPressuredTime_%', 'SourcenumRecordsInPerSecond_avg',
       'SourcenumRecordsInPerSecond_std', 'SourcenumRecordsOutPerSecond_avg',
       'SourcenumRecordsOutPerSecond_std', 'SourcebusyTimeMsPerSecond_avg',
       'SourcebusyTimeMsPerSecond_std',
       'SourcebackPressuredTimeMsPerSecond_avg',
       'SourcebackPressuredTimeMsPerSecond_std', 'SourcebusyTime_%',
       'SourcebackPressuredTime_%', 'MappernumRecordsInPerSecond_avg',
       'Mapper

,i,rate,dvfs,SourcenumRecordsOutPerSecond_avg,SourcenumRecordsOutPerSecond_std,MapperbusyTimeMsPerSecond_avg,watts_avg,rxBytes,txBytes
0,0,300000,1.2,210849.815794,329.809767,977.533333,66.57,63154605332,25343640657
1,0,300000,1.3,207847.704484,712.379403,977.285417,69.72,61932382482,24841090430
2,0,300000,1.4,261863.456190,624.233204,982.256250,73.43,78085383874,31324747154
3,0,300000,1.5,247732.336508,1217.538177,862.191667,73.04,73450324385,29475130343
4,0,300000,1.6,270013.847222,3849.388836,900.460417,76.93,80015632328,32111119016
5,0,300000,1.7,299957.071270,103.421129,897.233333,80.02,91708821390,36765820061
6,0,300000,1.8,299976.737738,78.631535,877.325000,82.13,88827125268,35624341614
7,0,300000,1.9,300010.039563,152.381924,900.700000,88.11,89119594363,35740110935
8,0,300000,2.0,299988.397421,10.659872,809.302083,87.06,88963000929,35660519817
9,0,300000,2.1,299999.316786,97.437475,765.391667,86.94,88331113282,35399814234


In [29]:
print(132810/24841090430)

5.346383661145909e-06
